In [1]:
import pandas as pd
from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from datasets import Dataset

c:\Users\Beni\anaconda3\envs\is_edita\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# read the data
df = pd.read_csv('processed_data.csv')
df
# shuffle the data with seed 42
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
# turn short_description to string
df['short_description'] = df['short_description'].astype(str)
df["headline"] = df["headline"].astype(str)
df["authors"] = df["authors"].astype(str)
# numerically encode the category column
df['category'] = pd.Categorical(df['category']).codes
df["text"] = df["short_description"] + " " + df["headline"] + " " + df["authors"]
num_categories = len(df['category'].unique())

# split the data into train and test
train_df = df[:int(len(df)*0.8)][:10000]
test_df = df[int(len(df)*0.8):][:10000]

# from dataframes to datasets (feature: text, label: category)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)



In [3]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_categories)
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=50,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    save_steps=500,
    evaluation_strategy='steps',
    eval_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# compute F1 on test
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {"accuracy": accuracy_score(labels, preds),
            "precision": precision_score(labels, preds, average='macro'),
            "recall": recall_score(labels, preds, average='macro'),
            "f1": f1_score(labels, preds, average='macro')}


In [5]:


# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the text data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Define a function to format the dataset correctly
def format_dataset(dataset):
    dataset = dataset.map(lambda examples: {'labels': examples['category']}, batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    return dataset

# Format the datasets
formatted_train_dataset = format_dataset(tokenized_train)
formatted_test_dataset = format_dataset(tokenized_test)


Map: 100%|██████████| 10000/10000 [00:00<00:00, 344781.71 examples/s]


In [6]:

# Update the Trainer with the formatted datasets
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=formatted_train_dataset,  
    eval_dataset=formatted_test_dataset,  
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

  1%|          | 10/1250 [00:05<08:40,  2.38it/s]

{'loss': 2.7165, 'learning_rate': 1e-05, 'epoch': 0.02}


  2%|▏         | 20/1250 [00:09<08:12,  2.50it/s]

{'loss': 2.6749, 'learning_rate': 2e-05, 'epoch': 0.03}


  2%|▏         | 30/1250 [00:13<08:07,  2.50it/s]

...

 99%|█████████▉| 1240/1250 [10:50<00:04,  2.47it/s]

{'loss': 0.334, 'learning_rate': 4.1666666666666667e-07, 'epoch': 1.98}


100%|██████████| 1250/1250 [10:54<00:00,  1.91it/s]

{'loss': 0.4044, 'learning_rate': 0.0, 'epoch': 2.0}
{'train_runtime': 654.4479, 'train_samples_per_second': 30.56, 'train_steps_per_second': 1.91, 'train_loss': 0.7685119585037231, 'epoch': 2.0}


TrainOutput(global_step=1250, training_loss=0.7685119585037231, metrics={'train_runtime': 654.4479, 'train_samples_per_second': 30.56, 'train_steps_per_second': 1.91, 'train_loss': 0.7685119585037231, 'epoch': 2.0})

In [7]:
# evaluate the model
trainer.evaluate()


100%|██████████| 625/625 [01:21<00:00,  7.64it/s]


{'eval_loss': 0.5769881010055542,
 'eval_accuracy': 0.8382,
 'eval_precision': 0.8062456684035972,
 'eval_recall': 0.7914781246120076,
 'eval_f1': 0.7971966493925249,
 'eval_runtime': 81.8989,
 'eval_samples_per_second': 122.102,
 'eval_steps_per_second': 7.631,
 'epoch': 2.0}

In [9]:
# save model
trainer.save_model("distilbert-base-uncased-trained")